In [2]:
#Importing required libraries:
import os
import requests
import pandas as pd
import pytz  # For timezone handling
from datetime import datetime, timedelta
import warnings
from urllib3.exceptions import InsecureRequestWarning
import json
from gtfs-realtime-bindings import gtfs_realtime_pb2

SyntaxError: invalid syntax (885547685.py, line 10)

In [2]:
# Read the routes.txt file
file_path = "/workspaces/DRT_App/Assets/Data/Routes.txt" 
cr_df = pd.read_csv(file_path)
cr_df.head(20)

,route_id,incoming_bus,route_name
0,101,101,101. Route 101
1,112,112,112. Route 112
2,118,118,118. Route 118
3,121,121,121. Route 121
4,211,211,211. Route 211
5,216,216,216. Route 216
6,224,224,224. Route 224
7,227,227,227. Route 227
8,301,301,301. Route 301
9,302,302,302. Route 302


In [ ]:
GTFS_URL = "https://drtonline.durhamregiontransit.com/gtfsrealtime/VehiclePositions"

# Fetch the GTFS-Realtime feed
response = requests.get(GTFS_URL)
if response.status_code != 200:
    print(f"❌ Failed to fetch data. HTTP Status: {response.status_code}")
    exit()

# Parse GTFS-Realtime feed
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

# Convert to DataFrame
vehicles = []
for entity in feed.entity:
    if entity.HasField("vehicle"):
        vehicle = entity.vehicle
        vehicles.append({
            "id": vehicle.vehicle.id,
            "route_id": vehicle.trip.route_id,
            "latitude": vehicle.position.latitude,
            "longitude": vehicle.position.longitude,
            "speed": vehicle.position.speed if vehicle.position.HasField("speed") else None,
            "timestamp": vehicle.timestamp
        })

df = pd.DataFrame(vehicles)
df.head()  # Show first few rows

if vehicles_data is None:
    print("❌ No data fetched. Please check the network connection or the API endpoint.")
else:
    print("✅ Vehicles data fetched successfully!")

# Function to process and filter Vehicles data
def process_vehicles_data(data, concat):
    vehicles = []

    for entity in data.get("entity", []):
        vehicle_info = entity.get("vehicle", {})
        trip_info = vehicle_info.get("trip", {})
        position_info = vehicle_info.get("position", {})

        # Extract trip details
        trip_id = trip_info.get("trip_id")
        route_id = trip_info.get("route_id")
        direction_id = trip_info.get("direction_id")
        start_time = trip_info.get("start_time")
        start_date = trip_info.get("start_date")
        schedule_relationship = trip_info.get("schedule_relationship")
        vehicle_id = vehicle_info.get("vehicle", {}).get("id")
        latitude = position_info.get("latitude")
        longitude = position_info.get("longitude")

        # Skip if not scheduled
        if schedule_relationship != "SCHEDULED":
            continue

        # Combine route_id and direction_id
        combined_id = str(route_id) + str(direction_id)

        # Filter by the provided 'concat' value
        if combined_id == concat:
            vehicles.append({
                "trip_id": trip_id,
                "route_id": route_id,
                "direction_id": direction_id,
                "vehicle_id": vehicle_id,
                "start_time": start_time,
                "start_date": start_date,
                "latitude": latitude,
                "longitude": longitude
            })

    return pd.DataFrame(vehicles)

# Example 'concat' value (update this to match your filter)
concat = '4486_881810'  # Example: route_id + direction_id

# Process and filter the data
if vehicles_data:
    filtered_vehicles = process_vehicles_data(vehicles_data, concat).sort_values(by="start_time", ascending=True).reset_index(drop=True)

    if filtered_vehicles.empty:
        print("⚠️ No vehicles found for the specified 'concat' filter.")
    else:
        print("✅ Filtered vehicles data:")
        print(filtered_vehicles)

Raw response content:


2.0 ����Z
6109 "P
0
 1205__491002_Timetable_-_2025-0220250323*N1
Vn/B���  (����B
6109Z
6140 "P
0
 1245__291003_Timetable_-_2025-0220250323*N1
��/B���  (����B
6140Z
7120 "P
0
 1246__291002_Timetable_-_2025-0220250323*N1
��/BĶ��  (����B
7120Z
7124 "P
0
 1216__491004_Timetable_-_2025-0220250323*N1
�'/BPc��  (����B
7124Z
8512 "P
0
 1195__291001_Timetable_-_2025-0220250323*N1
�d/B�%��  (����B
8512Z
8573 "P
0
 1287__4910
Request Error: Expecting value: line 3 column 1 (char 3)
❌ No data fetched. Please check the network connection or the API endpoint.
